In [ ]:
#-----------------------------------* ATTENTION ATTEMPT *---------------------------------------
# Implementation of a model inspired by the paper "A Transformer-based Framework for Multivariate Time Series Representation Learning", 
# link: https://dl.acm.org/doi/10.1145/3447548.3467401

# Problem description: develop a forecasting model that is able to predict several uncorrelated time series

# DATA STRUCTURE: 
# Single folder containing the following files:
# -> 'training_data.npy': it contains a numpy array of shape (48000, 2776). 48000 time series of length 2776.
# -> 'valid_periods.npy': it contains a numpy array of type (48000, 2) containing for each of the time series the start and end index of the current series, i.e. the part without padding.
# -> 'categories.npy': it contains a numpy array of shape (48000,), containing for each of the time series the code of its category. The possible categories are in {'A', 'B', 'C', 'D', 'E', 'F'}.
# IMPORTANT: This is a dataset consisting of monovariate time series, i.e. composed of a single feature, belonging to six different domains. The time series of each domain are not to be understood as closely related to each other, but only as collected from similar data sources.
# What is required of you is therefore to build a model that is capable of generalising sufficiently to predict the future samples of the 60 time series of the test set.

In [ ]:
# Import libraries and connect to drive personal folder

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/homework_2/Edoardo

# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from keras import backend as K
from keras.callbacks import TensorBoard
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)


# Import other support libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler


In [ ]:
# Define function for data inport:
# The function takes as input the path of the folder containing the data and returns the training data, the validation periods and the categories
# Before doing so, it eventually unzips it if the zip flag is set to True

def import_data(path, zip=False):
    if zip:
        !unzip -q $path
    training_data = np.load('training_data.npy')
    valid_periods = np.load('valid_periods.npy')
    categories = np.load('categories.npy')
    return training_data, valid_periods, categories

# Call the function to import the data
training_data, valid_periods, categories = import_data('training_dataset.zip', zip=False)

# Check the shape of the data
print(training_data.shape)
print(valid_periods.shape)
print(categories.shape)

# Transform the data by extending the dimensions (number of time series, length of time series, number of features)
training_data_no_pad = np.expand_dims(training_data, axis=2)

In [ ]:
# -----------------------------------* DATA ANALYSIS *---------------------------------------

# Note that all the values are store as float64, so we can convert them to float32 to save memory
if training_data.dtype == np.float64:
    training_data = training_data.astype(np.float32)

# Then transform the categories into numbers
categorical_to_numerical = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F':5}

if categories[0] == "D":
    for category in np.unique(categories):
        categories[categories == category] = categorical_to_numerical[category]
    # To save memory, convert the categories to int32
    categories = categories.astype(np.int32)

# -<-<-<-<-<-<-<-<-<-< FIRST PLOTS AND PADDING REMOVAL/NORMALIZATION >->->->->->->->->->->-

# Plot the first 10 time series

fig, axs = plt.subplots(2, 5, figsize=(20, 10))
axs = axs.ravel()
for i in range(10):
    axs[i].plot(training_data[i])
    axs[i].set_title('Category: {}'.format(categories[i]), fontsize=10)
    axs[i].set_xlabel('Time', fontsize=10)
    axs[i].set_ylabel('Value', fontsize=10)

fig.suptitle('First 10 time series', fontsize=30)

# -<-<-<-<-<-<-<-<-<-<-<-< DATA NORMLIZATION >->->->->->->->->->->->->-
# AS THE DATA HAS ALREADY BEEN NORMALIZED (EACH ROW HAS BEEN NORMALIZED INDIPENDENTLY)
# WE DEEMED THIS STEP NOT NECESSARY

# Define min_max normalization function
def min_max_norm(data, min, max):
    return (data - min) / (max - min)

# Apply the function over the categories singularly
def min_max_norm_by_category(data, categories):
    # Loop over the categories
    for category in np.unique(categories):
        # For each category compute the min and max
        C_min = np.min(data[categories == category])
        C_max = np.max(data[categories == category])
        # Apply the min_max_norm function to the data of the current category
        data[categories == category] = min_max_norm(data[categories == category], C_min, C_max)
    return data

# Function to apply a robust scaler on each non-padded time series
# The function takes as input the data and loops over each time series, applying the robust scaler to each of them
def robust_scaler_normalization(data):
    # Loop over the time series
    for i in range(data.shape[0]):
        # Apply the robust scaler to the current time series
        data[i, :, 0] = RobustScaler().fit_transform(data[i, :, 0])
    return data


# -<-<-<-<-<-<-<-<-<-<  PADDING REMOVAL >->->->->->->->->->-
# Define a function to remove the padding from the time serie

def remove_padding(data, valid_periods):
    data_no_pad = []
    for i in range(data.shape[0]):
        data_no_pad.append(data[i, valid_periods[i, 0]:valid_periods[i, 1]])
    return np.array(data_no_pad)


# Remove the padding from the time series
training_data_no_pad = remove_padding(training_data_no_pad, valid_periods)

# Call the function for robust scaler normalization
training_data_no_pad = robust_scaler_normalization(training_data_no_pad, categories)

# Plot the first 10 time series without padding, keeping the information about the orignal temporal location
fig, axs = plt.subplots(2, 5, figsize=(20, 10))
axs = axs.ravel()
for i in range(10):
    axs[i].plot(range(valid_periods[i, 0], valid_periods[i, 1]), training_data_no_pad[i])
    axs[i].set_title('Category: {}'.format(categories[i]), fontsize=10)
    axs[i].set_xlabel('Time', fontsize=10)
    axs[i].set_ylabel('Value', fontsize=10)

fig.suptitle('First 10 time series without padding and normalized', fontsize=30)

# All this samples belong to category D, and are "to be understood as not closely related to each other, but only as collected from similar data sources"
# This means that the time seies within the same categoty are to be considered as uncorrelated, but should there not be a correlation between the catoegories?



In [ ]:

# -<-<-<-<-<-<-<-<-<-< DATA DISTRIBUTION ANALYSIS >->->->->->->->->->->-


# GROUP DATA BY CATEGORY:

# Transform the categories into numbers
categorical_to_numerical = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F':5}

if categories[0] == "D":
    for category in np.unique(categories):
        categories[categories == category] = categorical_to_numerical[category]
    # To save memory, convert the categories to int32
    categories = categories.astype(np.int32)

# Now plot the distribution of the time series over the different categories
fig, ax = plt.subplots(figsize=(20, 10))
ax.hist(categories, bins=6)
ax.set_title('Distribution of the time series over the different categories', fontsize=20)
ax.set_xlabel('Category')
ax.set_ylabel('Number of time series')
# separate the bars and use different colors
for i in range(6):
    ax.patches[i].set_color('C{}'.format(i))
plt.xticks(np.arange(6))

# OSS: a visible imbalance between the categories is present, with category 5 been extremely underrepresented 
# (category 1 is just slightly underrepresented, while the other categories are more or less equally represented)

# Compute class_weight paramenter to be used for the fit of the model
weights = {}
for i in range(6):
    weights[i] = 1 / np.sum(categories == i)
print(weights)


# -<-<-<-<-<-<-< PLOT OF the FIRST TIME SERIES FOR DIFFERENT CATEGORIES >->->->->->-
# Plot the first n time series for each category (6 diffrerent plots, with multiple time series in each plot)

n = 10
fig, axs = plt.subplots(6, 1, figsize=(20, 30))
for i in range(6):
    axs[i].plot(training_data_no_pad[categories == i][:n].T)
    axs[i].set_title('Category: {}'.format(i), fontsize=20)
    axs[i].set_xlabel('Time', fontsize=20)
    axs[i].set_ylabel('Value', fontsize=20)

# -<-<-<-<-<-<-< PLOT OF TIME SERIES BY LENGHT AND DOMAIN  >->->->->->-

# Compute mean lenght of the time series for each category and plot them in a bar plot
mean_lenght = []
for i in range(6):
    mean_lenght.append(np.mean(valid_periods[categories == i, 1] - valid_periods[categories == i, 0]))
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(np.arange(6), mean_lenght)
ax.set_title('Mean lenght of the time series for each category', fontsize=20)
ax.set_xlabel('Category')
ax.set_ylabel('Mean lenght')
plt.xticks(np.arange(6))
for i in range(6):
    ax.patches[i].set_color('C{}'.format(i))

# OSS: a fairly similar mean lenght for all the categories




In [ ]:
# Define some Hyperparameters for the forcasting model
WINDOW_SIZE = 200
BATCH_SIZE = 256
STRIDE = 10
EPOCHS = 100
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.03
TELESCOPE = 18
AUTOREGRESSIVE_TELESCOPE = 3
assert AUTOREGRESSIVE_TELESCOPE < TELESCOPE

# In this case, given the nature of the problem, as well as the fact that the time series are not correlated
# (so introducing some of them in training phase should now bias the test), we decide
# to make use of the great variaty of time series available, and split data among the time series themselves:

# This way, a certain percentage of the time series will be used for training (80% initial partition), while the remaining ones will be used for validation (20).
# In both case, we define the number of samples to be predicted as TELESCOPE, and the number of samples to be used for the prediction as WINDOW_SIZE.

# For the testing, we respect the imbalances among the different domains using stratify based on the categories

X_train_raw, X_test_raw, domains_train, domains_test = train_test_split(training_data_no_pad, categories, test_size=TEST_SPLIT, stratify=categories)
X_train_raw, X_val_raw, domains_train, domains_val = train_test_split(X_train_raw, domains_train, test_size=VALIDATION_SPLIT, stratify=domains_train)

In [ ]:
# -----------------------------------* DATA PREPROCESSING *---------------------------------------
# Build the sequences for the forecasting model: in this case, we will try to take into account the 
# imbalance between the categories usign a weighted loss function

# The function lets us build the sequences to use for the training of the forecasting model:
# In this case, given a certain domain (category), the function will build the sequences to be used for the training
# by extracting temporal windows out of each time series.
# Please note tha some of the time series will takes as they are, since they are shorter than the window size.
# In some other cases, the time series are "partioned" in windows of the same size, and the last window is padded with zeros
# In any case, for the correct creation of the sequences, dataset to be passed to the function must be the one without padding
def build_sequences_on_domain(data, categories_split  ,domain = 0, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    padding_mask = []
    temp_df = data[categories_split == domain]
    temp_label = temp_df.copy()
    # For each time series of the specific domain, compute the padding lenght
    # At the same time, generate the padding mask that should have dimension (batch_size, time_series_lenght):
    # Ideally we generate a mask for lenght = time_series_lenght + padding_lenght for each signal, then
    # we generate the padding mask during the generation of sequences using the masks previously generated
    # OSS... This is done on the data without padding
    masks = []
    for time_series_n in range(len(temp_df)):
        padding_len = (window + telescope) - len(temp_df[time_series_n])
        if(padding_len > 0):
            padding = np.zeros((padding_len,1), dtype='float32')
            temp_df[time_series_n] = np.concatenate((padding,temp_df[time_series_n]))
            padding = np.zeros((padding_len,1), dtype='float32')
            temp_label[time_series_n] = np.concatenate((padding,temp_label[time_series_n]))
            # Generate the padding mask
            masks[time_series_n] = np.concatenate((np.zeros((padding_len), dtype='int32'), np.ones((len(temp_df[time_series_n])-padding_len), dtype='int32')))
            
        
        
    # Generate the sequences and proper padding masks
    for time_series_n in range(len(temp_df)):
          for i in np.arange(0, len(temp_df[time_series_n])-window-telescope, stride):
              dataset.append(temp_df[time_series_n][i:i+window])
              padding_mask.append(masks[time_series_n][i:i+window])
              labels.append(temp_label[time_series_n][i+window:i+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

def inspect_multivariate(X, y, telescope, idx=None, data_to_plot= 5):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    # Plot three sequences chosen based on idx
    figs, axs = plt.subplots(data_to_plot, 1, sharex=True, figsize=(30,15))
    for i in range(idx, idx+data_to_plot):
        axs[i-idx].plot(np.arange(len(X[i])), X[i])
        axs[i-idx].scatter(np.arange(len(X[i]), len(X[i])+telescope), y[i], color='orange')
        axs[i-idx].set_title('Sequence {}'.format(i))
        axs[i-idx].set_ylim(0,1)
    plt.show()


def inspect_multivariate_prediction(X, y, pred, telescope, idx=None, data_to_plot= 5):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    # Plot data_to_plot sequences chosen based on idx
    figs, axs = plt.subplots(data_to_plot, 1, sharex=True, figsize=(30,15))
    for i in range(idx, idx+data_to_plot):
        axs[i-idx].plot(np.arange(len(X[i])), X[i])
        axs[i-idx].scatter(np.arange(len(X[i]), len(X[i])+telescope), y[i], color='orange')
        axs[i-idx].scatter(np.arange(len(X[i]), len(X[i])+telescope), pred[i], color='green')
        axs[i-idx].set_title('Sequence {}'.format(i))
        axs[i-idx].set_ylim(0,1)
    plt.show()

In [ ]:
X_train, y_train = build_sequences_on_domain(data=X_train_raw,categories_split=domains_train, domain = 0, window=WINDOW_SIZE, stride=STRIDE, telescope=AUTOREGRESSIVE_TELESCOPE)
X_val, y_val = build_sequences_on_domain(data=X_val_raw,categories_split = domains_val,domain=0, window=WINDOW_SIZE, stride=STRIDE, telescope=AUTOREGRESSIVE_TELESCOPE)
X_test_reg, y_test_reg = build_sequences_on_domain(data=X_test_raw,categories_split = domains_test,domain=0, window=WINDOW_SIZE, stride=STRIDE, telescope=TELESCOPE)
for i in range(1,6):
    X_train_temp, y_train_temp = build_sequences_on_domain(data=X_train_raw,categories_split=domains_train, domain=i, window=WINDOW_SIZE, stride=STRIDE, telescope=AUTOREGRESSIVE_TELESCOPE)
    X_val_temp, y_val_temp = build_sequences_on_domain(data=X_val_raw,categories_split = domains_val,domain=i, window=WINDOW_SIZE, stride=STRIDE, telescope=AUTOREGRESSIVE_TELESCOPE)
    X_test_temp, y_test_temp = build_sequences_on_domain(data=X_test_raw,categories_split = domains_test,domain=i, window=WINDOW_SIZE, stride=STRIDE, telescope=TELESCOPE)
    # Concatenate with the previous domains
    X_train = np.concatenate((X_train, X_train_temp))
    y_train = np.concatenate((y_train, y_train_temp))
    X_val = np.concatenate((X_val, X_val_temp))
    y_val = np.concatenate((y_val, y_val_temp))
    X_test_reg = np.concatenate((X_test_reg, X_test_temp))
    y_test_reg = np.concatenate((y_test_reg, y_test_temp))


In [ ]:
inspect_multivariate(X_train, y_train, AUTOREGRESSIVE_TELESCOPE)

In [ ]:
# Define input and output shape for the forecasting model

print(X_train.shape)
print(X_val.shape)
print(X_test_reg.shape)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

print(input_shape)
print(output_shape)

In [ ]:


# -----------------------------------* FORECASTING MODEL DEFINITION *---------------------------------------

# Define the forecasting model
# The model is composed of a first encoder, that, in our case (being a univariate prediction problem) is composed of a single Conv1D layer
# with 1 input size and d output size, where d is the number of filters (and the dimension of the latent space of the input embedding).
# Each kernel K_i (i = 1, ... , d) has a a dimension of (k,m), where 𝑘 is the width in number of time steps and m is the number of features(variables, so 1 in this case).
# Before sending the embedded input into the transformer, we add a positional encoding to the input: for this purpose, we define two different possible encodings:
# -> a fixed encoding, based on sinuosoidal functions
# -> a learned encoding, that is learned during the training phase
# As proposed in the paper, we generate a padding mask which adds a large negative value to the attention scores for the padded positions in the signals whose valid lenght
# is smaller than the window size.
# Finally, the transformer is a simple FFN with batch normalization and dropout.

# -<-<-<-<-<-<-<-<-<-<-<- POSITIONAL ENCODING >->->->->->->->->->->->->-

class FixedPositionalEncoding(tfk.Layer):
    # d_model : dimension of the latent space of the input embedding
    # dropout_rate : dropout rate to be applied to the input
    # max_len : maximum lenght of the time series
    def __init__(self, d_model, dropout_rate = 0.1, max_len=200, scale_factor=1.0):
        super(FixedPositionalEncoding, self).__init__()
        self.dropout = dropout_rate
        # Compute the positional encoding vector for the given max_len (maximul lenght of the time series)
        # for the given d_model (dimension of the latent space of the input embedding).
        # For the i-th position of the input, the positional encoding is computed as:
        pe = tf.zeros((max_len, d_model), dtype=tf.float32)
        position = tf.range(0, max_len, dtype=tf.float32)[:, tf.newaxis]
        div_term = tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / d_model))
        pe[:, 0::2] = tf.sin(position * div_term)
        pe[:, 1::2] = tf.cos(position * div_term)
        pe = scale_factor * tf.expand_dims(pe, axis=0)
        self.pe = tf.Variable(pe, trainable=False)

    def call(self, x):
        x = x + self.pe[:tf.shape(x)[0], :]
        x = tfkl.Dropout(rate=self.dropout)(x)
        return x


class LearnablePositionalEncoding(tfk.Layer):

    # Define the position encoding as trainable parameters
    def __init__(self, d_model,dropout_rate = 0.1, max_len=1024):
        super(LearnablePositionalEncoding, self).__init__()
        self.dropout = dropout_rate
        self.pe = tf.Variable(tf.random.uniform((max_len, 1, d_model), minval=-0.02, maxval=0.02))

    def call(self, x):
        x = x + self.pe[:tf.shape(x)[0], :]
        x = tfkl.Dropout(rate=self.dropout)(x)
        return self.dropout(x)
    

# -<-<-<-<-<-<-<-<-<-<-<- TRANSFORMER DEFINITION >->->->->->->->->->->->->-

# Define the transformer encoder layer with batch normalization instead of layer normalization

class TransformerLayer(tfkl.Layer):
    # d_model : dimension of the latent space of the input embedding
    # nhead : number of heads in the multihead attention mechanism
    # dim_feedforward : dimension of the feedforward network
    # dropout : dropout rate to be applied in the net
    # activation : activation function to be applied in the net
    def __init__(self, d_model, nhead = 4, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerLayer, self).__init__()
        # Define all the layers in the model
        self.self_attn = tfkl.MultiHeadAttention(d_model, nhead, dropout=dropout)
        self.dropout1 = tfkl.Dropout(rate=dropout)
        self.batch_norm1 = tfkl.BatchNormalization(epsilon=1e-5)
        self.dense1 = tfkl.Dense(dim_feedforward, activation=activation)
        self.dropout2 = tfkl.Dropout(rate=dropout)
        self.dense2 = tfkl.Dense(d_model)
        self.dropout3 = tfkl.Dropout(rate=dropout)
        self.batch_norm2 = tfkl.BatchNormalization(epsilon=1e-5)

    def call(self, embedded_output, attention_mask=None,):
        # Connect the different initialized layers:
        # embedded_output : input-encoder output to be fed to the transformer
        # attention_mask : mask to be applied to the attention layer to mask the padded positions
        src = embedded_output
        # -----* MULTIHEAD ATTENTION *-----
        # Multiheaded, self-attention mechanism
        src2 = self.self_attn(src, src, attention_mask=attention_mask)
        # Apply dropout
        src2 = self.dropout1(src2)
        # -----* RESIDUAL CONNECTION AND BATCH NORMALIZATION *-----
        # Residual connection
        src = src + src2
        # Apply batch normalization
        src = self.batch_norm1(src)
        # -----* FEEDFORWARD NETWORK *-----
        # Apply first dense layer
        src2 = self.dense1(src)
        # Apply dropout
        src2 = self.dropout2(src2)
        # Apply second dense 
        src2 = self.dense2(src2)
        # Apply dropout
        src2 = self.dropout3(src2)
        # Residual connection
        src = src + src2
        # Apply batch normalization
        src = self.batch_norm2(epsilon=1e-5)(src)
        return src
    
# Generate a class to stack multiple transformer layers
class TransformerStack(tfkl.Layer):
    def __init__(self, d_model, nhead = 4, dim_feedforward=2048, dropout=0.1, activation="relu", layers=1):
        super(TransformerStack, self).__init__()
        self.layers = [TransformerLayer(d_model, nhead, dim_feedforward, dropout, activation) for _ in range(layers)]
    
    def call(self, embedded_output, attention_mask=None):
        # Apply the attentoion mask to the first layer
        src = self.layers[0](embedded_output, attention_mask)
        # Apply the remaining layers
        for layer in self.layers[1:]:
            src = layer(src)
        return src

# -<-<-<-<-<-<-<-<-<-<-<- FORECASTING MODEL DEFINITION >->->->->->->->->->->->->-
# The following function defines the forecasting model:
def build_MVTSForecasting_model(input_shape,output_shape, padding_mask,encoder = 'conv1d',fixed_pos_encoding=False,, layers=1):
    # Ensure the input time steps are at least as many as the output time steps
    assert input_shape[0] >= output_shape[0], "We want input time steps to be >= of output time steps"

    # Define some hyperparameters
    max_len = input_shape[0]
    d_model = 64
    nhead = 4
    dim_feedforward = 2048
    dropout_rate_encoding = 0.1
    dropout_rate_transformer = 0.1

    # Defore using the padding mask, we need to reshape it
    mask = tf.cast(tf.expand_dims(padding_mask, axis=1), "int32")

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Define the encoder layer
    # The encoder is composed of a single Conv1D layer with 1 input size and d output size, where d is the number of filters (and the dimension of the latent space of the input embedding).
    # Each kernel K_i (i = 1, ... , d) has a a dimension of (k,m), where 𝑘 is the width in number of time steps and m is the number of features(variables, so 1 in this case).

    if encoder == 'conv1d':
        encoder = tfkl.Conv1D(filters=d_model, kernel_size=5, strides=1, padding='same', name='encoder')(input_layer)
    elif encoder == 'dense':
        encoder = tfkl.Dense(d_model, activation='relu', name='encoder')(input_layer)

    # Define the positional encoding layer
    if fixed_pos_encoding:
        pos_encoder = FixedPositionalEncoding(d_model, dropout_rate=dropout_rate_encoding, max_len=max_len, scale_factor=1.0)(encoder)
    else:
        pos_encoder = LearnablePositionalEncoding(d_model, dropout_rate=dropout_rate_encoding, max_len=max_len)(encoder)

    # Proceed with the transformer layers
    transformer = TransformerStack(d_model, nhead, dim_feedforward, dropout_rate_transformer, layers=layers)(pos_encoder, mask)
    
    # Add non-linear activation function
    end = tfkl.Activation('relu')(transformer)

    # Define dropout layer
    end = tfkl.Dropout(0.1)(end)

    output = output * tf.expand_dims(mask, axis=-1)

    #Finally, the output layer is a simple dense layer with AUTOREGRESSIVE_TELESCOPE output size
    output_layer = tfkl.Dense(output_shape[0], name='output_layer')(end)

    # Construct the model by connecting input and output 
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model




# -----------------------------------* LR SCHEDULER  *---------------------------------------


def lr_warmup_cosine_decay(global_step,
                           warmup_steps,
                           hold = 0,
                           total_steps=0,
                           start_lr=0.0,
                           target_lr=1e-3):
    # Cosine decay
    learning_rate = 0.5 * target_lr * (1 + np.cos(np.pi * (global_step - warmup_steps - hold) / float(total_steps - warmup_steps - hold)))

    # Target LR * progress of warmup (=1 at the final warmup step)
    warmup_lr = target_lr * (global_step / warmup_steps)

    # Choose between `warmup_lr`, `target_lr` and `learning_rate` based on whether `global_step < warmup_steps` and we're still holding.
    # i.e. warm up if we're still warming up and use cosine decayed lr otherwise
    if hold > 0:
        learning_rate = np.where(global_step > warmup_steps + hold,
                                 learning_rate, target_lr)

    learning_rate = np.where(global_step < warmup_steps, warmup_lr, learning_rate)
    return learning_rate

# Define callback for learning rate scheduler and warmup

class WarmupCosineDecay(tfk.callbacks.Callback):
    def __init__(self, total_steps=0, warmup_steps=0, start_lr=0.0, target_lr=1e-3, hold=0):

        super(WarmupCosineDecay, self).__init__()
        self.start_lr = start_lr
        self.hold = hold
        self.total_steps = total_steps
        self.global_step = 0
        self.target_lr = target_lr
        self.warmup_steps = warmup_steps
        self.lrs = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = model.optimizer.lr.numpy()
        self.lrs.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = lr_warmup_cosine_decay(global_step=self.global_step,
                                    total_steps=self.total_steps,
                                    warmup_steps=self.warmup_steps,
                                    start_lr=self.start_lr,
                                    target_lr=self.target_lr,
                                    hold=self.hold)
        tfk.backend.set_value(self.model.optimizer.lr, lr)



total_steps = len(X_train)/BATCH_SIZE *EPOCHS

warmup_steps = int(0.05*total_steps)


# Print learning rate on tensorboard: write appropriate callback

class LRTensorBoard(TensorBoard):
    # add other arguments to __init__ if you need
    def __init__(self, log_dir, **kwargs):
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs.update({'lr': K.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)

In [ ]:
model = build_Att_biLSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data=(X_val, y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5)
    ],
    class_weight=weights
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()


model.save('Fifth_Attempt/FifthAttempt')

In [ ]:
del model

In [ ]:
model = tfk.models.load_model('Fifth_Attempt/FifthAttempt')
# Autoregressive Forecasting
reg_predictions = np.array([])
X_temp = X_test_reg
for reg in range(0,TELESCOPE,AUTOREGRESSIVE_TELESCOPE):
    pred_temp = model.predict(X_temp,verbose=0)
    if(len(reg_predictions)==0):
        reg_predictions = pred_temp
    else:
        reg_predictions = np.concatenate((reg_predictions,pred_temp),axis=1)
    X_temp = np.concatenate((X_temp[:,AUTOREGRESSIVE_TELESCOPE:,:],pred_temp), axis=1)


In [ ]:
# Print the shape of the predictions
print(f"Predictions shape: {reg_predictions.shape}")

# Calculate and print Mean Squared Error (MSE)
mean_squared_error = tfk.metrics.mean_squared_error(y_test_reg.flatten(), reg_predictions.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")

# Calculate and print Mean Absolute Error (MAE)
mean_absolute_error = tfk.metrics.mean_absolute_error(y_test_reg.flatten(), reg_predictions.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")

In [ ]:
inspect_multivariate_prediction(X_test_reg, y_test_reg,reg_predictions, TELESCOPE)